In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 原始数据转序列

In [2]:
#读取数据
filename = 'data/stay_one_community_2019.csv'
act_points2019 = pd.read_csv(filename,parse_dates=['t_start', 't_end'])
act_points2019['date'] = pd.to_datetime(act_points2019['date'],format='%Y-%m-%d')
act_points2019.head()

,pid,date,t_start,t_end,poi_id,community_id,ptype,longitude,latitude,week,weekday
0,90221,2019-05-06,2019-05-06 07:28:00,2019-05-06 22:51:56,1,2291,2,114.185092,22.651382,19,0
1,90221,2019-05-07,2019-05-07 00:52:01,2019-05-07 11:44:26,0,2291,1,114.174066,22.640383,19,1
2,90221,2019-05-07,2019-05-07 12:09:41,2019-05-07 13:39:58,0,2291,1,114.174066,22.640383,19,1
3,90221,2019-05-07,2019-05-07 13:41:28,2019-05-07 16:42:16,1,2291,2,114.185092,22.651382,19,1
4,90221,2019-05-07,2019-05-07 16:43:47,2019-05-07 23:14:57,0,2291,1,114.174066,22.640383,19,1


In [3]:
# 3ptype
#原始ptype：0-到访，1-居住，2-工作，3-出行，
#现在，令0-居住（基点），1-出行，2-工作，3-到访（即其他活动）
dict_ = {0:3, 1:0, 2:2}
act_points2019['ptype'] = act_points2019['ptype'].map(dict_)

### 选取5~8月数据

In [4]:
# 生成月份标签
act_points2019['month'] = [data.month for data in act_points2019['date']]

# 月份选取
act_points2019_58 = act_points2019.query('month in [5,6,7,8]')

In [ ]:
act_points2019_58.to_csv('data/act_201958.csv',index=False)

### 实施转换

In [5]:
#以周为单位构建长时间序列（周一到周日为一周）,采取将天序列拼接后再reshape的方法
#序列长度24*60*7=10080
def act_2_sequence_in_week(act):
    from datetime import datetime
    Sequence_in_week = []
    
    for weekday in range(7):
        
        act_day = act.query(f'weekday=={weekday}')
        sequence = np.zeros(1440)
        for index in range(len(act_day)):
            
            data = act_day.iloc[index,:]

            #考虑隔日的活动，如从23:00到次日1:00 → t_start，t_end与date比较，并进行截断，t_start->0:00，t_end->23:59
            if data['t_start'].day != data['date'].day:
                data['t_start'] = datetime(data['date'].year,data['date'].month,data['date'].day,0,0,0)
            elif data['t_end'].day != data['date'].day:
                data['t_end'] = datetime(data['date'].year,data['date'].month,data['date'].day,23,59,59)

            #根据时间段修改初始活动类型
            act_begin = data['t_start'].hour*60 + data['t_start'].minute
            act_end = data['t_end'].hour*60 + data['t_end'].minute
            sequence[act_begin:act_end] = [data['ptype'] for i in range(act_end - act_begin)]

            #根据相邻活动的时间段加入出行，对出行时长小于15min且出行连接的活动类型一样的要去掉
            if index < len(act_day)-1:
                #下一个活动的数据
                data1 = act_day.iloc[index+1,:]
                #定义出行时间
                trip_begin = data['t_end'].hour*60 + data['t_end'].minute
                trip_end = data1['t_start'].hour*60+data1['t_start'].minute

                if (trip_end - trip_begin < 15) & (data['ptype'] == data1['ptype']):
                    sequence[trip_begin:trip_end] = [data['ptype'] for i in range(trip_end - trip_begin)]
                else:
                    sequence[trip_begin:trip_end] = [1 for i in range(trip_end - trip_begin)]
        
        Sequence_in_week.append(sequence)
    
    Sequence_in_week = np.array(Sequence_in_week).reshape(10080)
    
    return Sequence_in_week

In [6]:
# 对个体和周序号分组求序列
trajectory_groups_in_week = act_points2019_58.groupby(['pid','week'],group_keys=False)
trajectory_in_week1 = trajectory_groups_in_week.apply(act_2_sequence_in_week).reset_index()

,pid,week,0
0,90221,19,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,90221,20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,90221,21,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,90221,22,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,90221,24,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [10]:
trajectory_in_week1 = trajectory_in_week1.rename(columns = {0: "week_trajectory"})
trajectory_in_week1.head()

In [13]:
len(trajectory_in_week1)

1335

In [14]:
# 删除整周在家的轨迹（自成一类，不需分析）
trajectory_out = trajectory_in_week1
for index,data in enumerate(trajectory_out['week_trajectory']):
    #print(data)
    if (~data.any()):
        trajectory_out = trajectory_out.drop(index = index)

trajectory_in_week1 = trajectory_out.reset_index(drop = True)

In [19]:
trajectory[['pid','week']].to_csv('pid_week_201958.csv',index=False)

## 沃什-傅立叶变换

In [ ]:
# wft, copy from chen et.al.
def wft(x,num):
    Ipower = np.zeros(15) #时间序列长度不大于2^15
    y = np.zeros(num)
    
    
    for i in range(num):
        IB = i
        IL=0
        
        while True:
            IBD = int(IB/2)
            Ipower[IL] = 0 if IB == (IBD * 2) else 1
                
            if IB == 0 or IBD == 0:
                break
            IB = IBD
            IL = IL + 1
        
        IP = 0
        IFAC = num
        for t1 in range(IL+1):
            IFAC = int (IFAC / 2)
            IP = int (IP + IFAC * Ipower[t1])
        
        y[IP] = x[i]
        
    
    x = y.copy()
    Iter = int(np.log2(num))
    for M in range(Iter):
        nump = 1 if M==0 else int(nump*2)
        Mnum = int (num / nump)
        Mnum2 = int(Mnum / 2)
        alph = 1
        for MP in range(nump):
            IB = int(MP * Mnum)
            
            for MP2 in range(Mnum2):
                mnum21 = int(Mnum2 + MP2 + IB) 
                IBA = int(IB + MP2)
                y[IBA] = x[IBA] + alph * x[mnum21]
                y[mnum21] = x[IBA] - alph * x[mnum21]

            alph = -alph
        
        r = np.power(num,-0.5)
        for i in range(num):
            x[i] = y[i] * r
        
    return y

In [ ]:
trajectory = trajectory_in_week1['week_trajectory']
dim = len(trajectory)
next2 = np.power(2,int(np.log2(dim)+1))
WFTs = []

# sequence = np.pad(trajectory[0],(0,next2-dim),'constant', constant_values=(0, 0))
# wft_seq = wft(sequence,next2)

for index,data in enumerate(trajectory):
    # 0 padding
    sequence = np.pad(data,(0,next2-dim),'constant', constant_values=(0, 0))
    wft_seq = wft(sequence,next2)
    WFTs.append(wft_seq)

In [ ]:
# 储存一下wft的结果（运行太久了，，，）
np.savetxt("data/wft_week_3ptype_2019_58.txt", WFTs)

## 基于子水平集的持续同调

In [ ]:
from teaspoon.TDA.SLSP import Persistence0D
from gtda.diagrams import Scaler

In [ ]:
WFTs = np.loadtxt('data/wft_week_3ptype_2019_58.txt')
len(WFTs)

In [ ]:
# 用sublevel set function对WFT作持续同调，提取0维birth-death点
PH_0dim = []

for index,data in enumerate(WFTs):
    feature_ind_1, feature_ind_2, persistenceDgm = Persistence0D(data) 
    #indices of birth times, indices of death times, points set
    X = np.zeros((len(persistenceDgm),3))
    if bool(len(persistenceDgm)):
        X[:,0:2] = persistenceDgm #有空数组就会报错
    X_3dim = X[np.newaxis,:,:]
    scaler = Scaler()
    persistenceDgm = scaler.fit_transform(X_3dim)
    PH_0dim.append(persistenceDgm)

In [ ]:
#可视化
dt = PH_0dim[0].reshape((PH_0dim[0].shape[1],3))
plt.plot(dt[:,0],dt[:,1],'ro')
#plt.plot([min(WFTs[0])*1e+24, max(WFTs[0])*1e+24], [min(WFTs[0])*1e+24, max(WFTs[0])*1e+24],'k--')
plt.plot([-2,2], [-2,2],'k--')

## 持续同调图中的特征提取

In [ ]:
from sklearn.pipeline import make_pipeline, make_union
from gtda.diagrams import PersistenceEntropy
from gtda.images import HeightFiltration
from gtda.diagrams import Amplitude

#Listing all metrics we want to use to extract diagram amplitudes
metric_list = [
    {"metric": "bottleneck", "metric_params": {}},
    {"metric": "wasserstein", "metric_params": {"p": 1}},
    {"metric": "wasserstein", "metric_params": {"p": 2}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 2, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 2, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 1, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 3.2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 3.2, "n_bins": 100}},
]

#
feature_union = make_union(
    *[PersistenceEntropy(nan_fill_value=-1)]
    + [Amplitude(**metric, n_jobs=-1) for metric in metric_list]
)

# tda_union = make_union(
#     *[make_pipeline(*diagram_step, feature_union) for diagram_step in diagram_steps],
#     n_jobs=-1
# )

In [ ]:
Features = []
for index,data in enumerate(PH_0dim):
    feature = feature_union.fit_transform(data)
    Features.append(feature)
    print(index)

In [ ]:
Features_reshape = np.array(Features).reshape(np.array(Features).shape[0],np.array(Features).shape[2])

In [ ]:
# 保存特征！！！
np.savetxt("data/persistence_features_week_2019_58.txt", Features_reshape)